# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [18]:
# Initial imports
import pandas as pd
import numpy as np
import random, os
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
# Create a connection to the database
username = os.getenv('postgres_username')
password = os.getenv('postgres_password')
server = os.getenv('postgres_server')
port = os.getenv('postgres_port')
database = os.getenv('postgres_database')
engine = create_engine(f"postgresql://{username}:{password}@{server}:{port}/{database}")

In [20]:
# Create cardholder table for reference
query = '''
        SELECT *
        FROM card_holder
        '''
cardholders = pd.read_sql(query, engine)
cardholders.head()

,cardholder_id,name
0,1,Robert Johnson
1,2,Shane Shaffer
2,3,Elizabeth Sawyer
3,4,Danielle Green
4,5,Sara Cooper


In [21]:
# Write function that locates outliers using standard deviation
def outliers_std(cardholder):
    std = np.std(cardholder['amount'])
    mean = np.mean(cardholder['amount'])
    outliers = cardholder[(cardholder['amount'] < (mean - 2 * std)) & (cardholder['amount'] > (mean + 2 * std))]
    cardholder_outliers_omitted = cardholder[(cardholder['amount'] > (mean - 2 * std)) & (cardholder['amount'] < (mean + 2 * std))]

In [22]:
# Find anomalous transactions for 3 random card holders
ids = random.sample(range(cardholders['cardholder_id'].min(), cardholders['cardholder_id'].max()), 3)
for id in ids:
    query = ('SELECT card_holder.cardholder_id, transaction.amount, transaction.date'
            ' FROM card_holder'
            '    INNER JOIN credit_card on card_holder.cardholder_id = credit_card.cardholder_id'
            '    INNER JOIN transaction on transaction.card_number = credit_card.card_number'
            f' WHERE card_holder.cardholder_id in ({id});')
    cardholder = pd.read_sql(query, engine)
    outliers_std(cardholder)

## Identifying Outliers Using Interquartile Range

In [39]:
# Write a function that locates outliers using interquartile range
def outliers_interquartile(cardholder):
    cardholder.sort_values('amount', axis=0, inplace=True)
    median = np.median(cardholder['amount'])
    Q1 = np.median(cardholder['amount'][:int(len(cardholder)/2)])
    Q3 = np.median(cardholder['amount'][int(len(cardholder)/2):])
    interquartile_range = Q3 - Q1
    inner_fence = [Q1 - interquartile_range * 1.5, Q3 + interquartile_range * 1.5]
    outer_fence = [Q1 - interquartile_range * 3, Q3 + interquartile_range * 3]
    minor_outliers = cardholder[(cardholder['amount'] < inner_fence[0]) | (cardholder['amount'] > inner_fence[1]) & (cardholder['amount'] > outer_fence[0]) | (cardholder['amount'] < outer_fence[1])]
    major_outliers = cardholder[(cardholder['amount'] < outer_fence[0]) | (cardholder['amount'] > outer_fence[1])]
    cardholder_wo_all_outliers = cardholder[(cardholder['amount'] > inner_fence[0]) & (cardholder['amount'] < inner_fence[1])]
    cardholder_wo_major_outliers = cardholder[(cardholder['amount'] > outer_fence[0]) & (cardholder['amount'] < outer_fence[1])]

In [40]:
# Find anomalous transactions for 3 random card holders
ids = random.sample(range(cardholders['cardholder_id'].min(), cardholders['cardholder_id'].max()), 3)
for id in ids:
    query = ('SELECT card_holder.cardholder_id, transaction.amount, transaction.date'
            ' FROM card_holder'
            '    INNER JOIN credit_card on card_holder.cardholder_id = credit_card.cardholder_id'
            '    INNER JOIN transaction on transaction.card_number = credit_card.card_number'
            f' WHERE card_holder.cardholder_id in ({id});')
    cardholder = pd.read_sql(query, engine)
    outliers_interquartile(cardholder)

     cardholder_id  amount                date
142             19    0.61 2018-09-21 07:53:08
194             19    0.78 2018-12-20 08:00:01
80              19    0.82 2018-05-24 00:29:49
108             19    1.13 2018-07-24 06:24:21
186             19    1.18 2018-12-02 15:46:59
55              19    1.21 2018-04-08 20:15:13
114             19    1.37 2018-07-29 06:02:10
134             19    1.39 2018-09-02 14:39:31
201             19    1.45 2018-12-26 19:55:23
46              19    1.52 2018-03-30 16:04:04
17              19    1.54 2018-02-01 21:52:43
37              19    1.56 2018-03-18 20:00:59
122             19    1.57 2018-08-09 06:00:26
87              19    1.62 2018-06-07 07:07:22
153             19    1.63 2018-10-05 22:12:25
133             19    1.65 2018-09-01 04:58:06
67              19    1.72 2018-04-29 08:03:32
100             19    1.75 2018-07-08 14:04:49
58              19    1.77 2018-04-09 19:04:52
167             19    1.86 2018-11-02 12:14:33
144          